In [1]:
# Import Libraries for Setup of Toronto Clustering / Coursera Capstone Week3
#Start of Week 3 Assignment

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import ssl
import csv

In [48]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

#print(soup.prettify())
print('soup ready')

soup ready


In [49]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()] 
#set up the initial table
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [50]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()
#Ignore cells with a borough that is Not assigned

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [51]:
df= df.groupby('PostalCode').agg(lambda x: ','.join(x))
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
df['Borough']= df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df.head()
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  Remove duplicate Bourough names.

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [52]:
df.shape
#shape of the final cleaned up table

(103, 2)

In [54]:
#Finish Week 3 Assignment Part 1


In [13]:
#Start Week 3 Assignment Part 2 
#Use Geocoder 
from  geopy.geocoders import Nominatim


In [55]:
#Get cooridiantes for Toronto
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [59]:
df1 = df.join(geocsv_data)
df1.head(20)
#join the two tables postal codes and coordinates

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [16]:
#End Week 3 Assignment Part 2 

In [17]:
#Start Week3 Assignment Part 3
#load relevant libraries
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library


In [60]:
#Generate neigborhoods data frame for Map
neighborhoods = df1
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [36]:
# create map of Toronto using latitude and longitude values
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
#map of Toronto using called Latitude and Longitude

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [69]:
#Cluster neigbourhoods with Scarborough in title
Toronto_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
neighborhoods2 = Toronto_data
neighborhoods2.head(20)

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476
5,Scarborough,Scarborough Village,43.744734,-79.239476
6,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [70]:
#map of Scarbourough using called Latitude and Longitude

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods2['Latitude'], neighborhoods2['Longitude'], neighborhoods2['Borough'], neighborhoods2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [71]:
# Finish Week 3 Part 3

In [218]:
#Start week 4 - 5 Capstone project  

Introduction to Capstone Project
Being a frequent traveler and often encountering delays one often looks for a bar to pass the time.  The business challenge we are looking to address is which airport in the Toronto area would most benefit by having a new bar established in proximty to the gates. As a bar entrepenuer we want to determine which airports already have a saturation of establishments and therefore perhaps do not support the concept of opening a new bar.  
The underlying assumption is if there is a concentration of establishments, then competition would be greater and thus a higher entry barrier to the market (Porter's forces).

Data Source
The Data Source for the analysis is the FourSquare data.  The locations of the two respective Neighborhood Airport locations, Toronto Pierseon, and Billy Bishop Airport are used for their cooridates.  The data of FourSquare is queired using the URL passing a requests for "Bar" within a proximate radius.  The results are filtered to eliminate extra fields and then mapped to indicate the density of establishments.  
See the charts in the notebook for details.

In [72]:
#Import Libraries to work with FourSquare Data
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [73]:
#Foursquare Credential Validation
CLIENT_ID = 'G3DE5BAPT5H2SY1TXS0JIU52M4ANQVSYWSZ4CGL2HGMIRIQF' # your Foursquare ID
CLIENT_SECRET = 'LX3T1NSCVOTF511NGDW4VSL2BKKB5IHIX3QOUEIGSJIU5WAD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G3DE5BAPT5H2SY1TXS0JIU52M4ANQVSYWSZ4CGL2HGMIRIQF
CLIENT_SECRET:LX3T1NSCVOTF511NGDW4VSL2BKKB5IHIX3QOUEIGSJIU5WAD


In [151]:
#Investigation of Bars near Billy Bishop Airport

address = '2 Eireann Quay, Toronto, ON M5V 1A1, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6322406 -79.3963494


In [160]:
#URL for the Foursquare request

search_query = 'Bar'
radius = 850

print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
url

Bar .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=G3DE5BAPT5H2SY1TXS0JIU52M4ANQVSYWSZ4CGL2HGMIRIQF&client_secret=LX3T1NSCVOTF511NGDW4VSL2BKKB5IHIX3QOUEIGSJIU5WAD&ll=43.6322406,-79.3963494&v=20180604&query=Bar&radius=850&limit=75'

In [161]:
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()



,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4d9e80a736818cfadff1f500,1 Stadium Rd 1st Floor,CA,Toronto,Canada,NaN,331,"[1 Stadium Rd 1st Floor, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.63324631070554...",43.633246,-79.400224,NaN,ON,Bar & Restaurant @ NYC,v-1579535853,NaN
1,"[{'id': '4bf58dd8d48988d14c941735', 'name': 'W...",False,4ade55bcf964a5201b7521e3,313 Bremner Blvd,CA,Toronto,Canada,Toronto CityPlace,1022,"[313 Bremner Blvd (Toronto CityPlace), Toronto...","[{'label': 'display', 'lat': 43.64040113723451...",43.640401,-79.390533,M5V 3V3,ON,St. Louis Bar and Grill,v-1579535853,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,57701594498e6daba644cceb,4 Eireann Quay,CA,Toronto,Canada,NaN,197,"[4 Eireann Quay, Toronto ON M5V 1A1, Canada]","[{'label': 'display', 'lat': 43.633702, 'lng':...",43.633702,-79.397748,M5V 1A1,ON,Aroma Espresso Bar,v-1579535853,NaN
3,"[{'id': '4bf58dd8d48988d156941735', 'name': 'M...",False,5e0e46b8d1a77000080d9706,NaN,CA,Toronto,Canada,NaN,1038,"[Toronto ON M5V 0C6, Canada]","[{'label': 'display', 'lat': 43.640661, 'lng':...",43.640661,-79.401907,M5V 0C6,ON,Makan Noodle Bar,v-1579535853,NaN
4,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,59889ae701559a531f82e23e,313 Bremner Blvd.,CA,Toronto,Canada,NaN,1027,"[313 Bremner Blvd., Toronto ON M5V 3V3, Canada]","[{'label': 'display', 'lat': 43.6404259, 'lng'...",43.640426,-79.390461,M5V 3V3,ON,St. Louis Bar & Grill,v-1579535853,NaN


In [162]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bar & Restaurant @ NYC,Bar,1 Stadium Rd 1st Floor,CA,Toronto,Canada,NaN,331,"[1 Stadium Rd 1st Floor, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.63324631070554...",43.633246,-79.400224,NaN,ON,4d9e80a736818cfadff1f500
1,St. Louis Bar and Grill,Wings Joint,313 Bremner Blvd,CA,Toronto,Canada,Toronto CityPlace,1022,"[313 Bremner Blvd (Toronto CityPlace), Toronto...","[{'label': 'display', 'lat': 43.64040113723451...",43.640401,-79.390533,M5V 3V3,ON,4ade55bcf964a5201b7521e3
2,Aroma Espresso Bar,Coffee Shop,4 Eireann Quay,CA,Toronto,Canada,NaN,197,"[4 Eireann Quay, Toronto ON M5V 1A1, Canada]","[{'label': 'display', 'lat': 43.633702, 'lng':...",43.633702,-79.397748,M5V 1A1,ON,57701594498e6daba644cceb
3,Makan Noodle Bar,Malay Restaurant,NaN,CA,Toronto,Canada,NaN,1038,"[Toronto ON M5V 0C6, Canada]","[{'label': 'display', 'lat': 43.640661, 'lng':...",43.640661,-79.401907,M5V 0C6,ON,5e0e46b8d1a77000080d9706
4,St. Louis Bar & Grill,Bar,313 Bremner Blvd.,CA,Toronto,Canada,NaN,1027,"[313 Bremner Blvd., Toronto ON M5V 3V3, Canada]","[{'label': 'display', 'lat': 43.6404259, 'lng'...",43.640426,-79.390461,M5V 3V3,ON,59889ae701559a531f82e23e
5,Organic Nail Bar,Nail Salon,26 Bathurst St,CA,Toronto,Canada,NaN,769,"[26 Bathurst St, Toronto ON M5V 0C6, Canada]","[{'label': 'display', 'lat': 43.638457, 'lng':...",43.638457,-79.400526,M5V 0C6,ON,577d8130498eeb45a9049858
6,Bobs bar,Beer Garden,NaN,CA,Toronto,Canada,NaN,585,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.636727, 'lng':...",43.636727,-79.400133,NaN,ON,5074da35e4b096e6d06b1009
7,Atlas Espresso Bar,Coffee Shop,"550 Front St. W, Unit 59",CA,Toronto,Canada,Portland,1012,"[550 Front St. W, Unit 59 (Portland), Toronto ...","[{'label': 'display', 'lat': 43.64106745249943...",43.641067,-79.399392,M5V 1C1,ON,530ecbf7498e72d8f20365d2
8,Macho Tex Mex Radio Bar,Mexican Restaurant,92 Fort York,CA,Toronto,Canada,CityPlace,864,"[92 Fort York (CityPlace), Toronto ON M5V 4A7,...","[{'label': 'display', 'lat': 43.63999631642935...",43.639996,-79.396980,M5V 4A7,ON,57228627498e3c0f95749614
9,Officers' Blue Barracks,Historic Site,NaN,CA,Toronto,Canada,NaN,943,"[Toronto ON M5V, Canada]","[{'label': 'display', 'lat': 43.639145, 'lng':...",43.639145,-79.403140,M5V,ON,5dfd199d9d34280008e9278e


In [163]:
#visualize all returns
dataframe_filtered.name

0                 Bar & Restaurant @ NYC
1                St. Louis Bar and Grill
2                     Aroma Espresso Bar
3                       Makan Noodle Bar
4                  St. Louis Bar & Grill
5                       Organic Nail Bar
6                               Bobs bar
7                     Atlas Espresso Bar
8                Macho Tex Mex Radio Bar
9                Officers' Blue Barracks
10    North And South Soldiers' Barracks
Name: name, dtype: object

In [164]:
#Plot of bars near airport
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Billy Bishop Airport',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [165]:
#Lets Compare the bar seen to Toronto Airport Neighborhood
#Investigation of Bars Near Airport

address = 'YYZ'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.678523999999996 -79.62912913064454


In [166]:
#URL for the Foursquare request

search_query = 'Bar'
radius = 850

print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
url

Bar .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=G3DE5BAPT5H2SY1TXS0JIU52M4ANQVSYWSZ4CGL2HGMIRIQF&client_secret=LX3T1NSCVOTF511NGDW4VSL2BKKB5IHIX3QOUEIGSJIU5WAD&ll=43.678523999999996,-79.62912913064454&v=20180604&query=Bar&radius=850&limit=75'

In [167]:
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4f3e41dde4b0fbbbd758f877,Terminal 3,CA,NaN,Canada,Toronto Pearson International Airport,853,[Terminal 3 (Toronto Pearson International Air...,"[{'label': 'display', 'lat': 43.68512516702658...",43.685125,-79.623741,NaN,Toast! Cafe & Bar,v-1579536756
1,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4bddbf1a6198c9b6ff8111ff,Toronto Pearson International Airport (YYZ),CA,Mississauga,Canada,Terminal 1,1020,[Toronto Pearson International Airport (YYZ) (...,"[{'label': 'display', 'lat': 43.67963051207084...",43.679631,-79.616551,ON,Hogtown Bar and Grill,v-1579536756
2,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,52572cf511d2b76cc7c5a19c,"Terminal 3, near Gate A13",CA,Mississauga,Canada,Toronto Pearson International Airport,858,"[Terminal 3, near Gate A13 (Toronto Pearson In...","[{'label': 'display', 'lat': 43.68522477548313...",43.685225,-79.623841,ON,Nobel Burger Bar,v-1579536756
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,589622930a3d54567eca1682,NaN,CA,Toronto,Canada,NaN,842,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.682519, 'lng':...",43.682519,-79.620245,ON,Ice Bar,v-1579536756
4,[],False,4d2cea9039a35481ad88b320,NaN,CA,Grand Case,Canada,NaN,822,"[Grand Case, Canada]","[{'label': 'display', 'lat': 43.684174, 'lng':...",43.684174,-79.622541,NaN,Sky's The Limit - Lolo's Bar & Restaurant,v-1579536756


In [168]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Toast! Cafe & Bar,Café,Terminal 3,CA,NaN,Canada,Toronto Pearson International Airport,853,[Terminal 3 (Toronto Pearson International Air...,"[{'label': 'display', 'lat': 43.68512516702658...",43.685125,-79.623741,NaN,4f3e41dde4b0fbbbd758f877
1,Hogtown Bar and Grill,Fast Food Restaurant,Toronto Pearson International Airport (YYZ),CA,Mississauga,Canada,Terminal 1,1020,[Toronto Pearson International Airport (YYZ) (...,"[{'label': 'display', 'lat': 43.67963051207084...",43.679631,-79.616551,ON,4bddbf1a6198c9b6ff8111ff
2,Nobel Burger Bar,Burger Joint,"Terminal 3, near Gate A13",CA,Mississauga,Canada,Toronto Pearson International Airport,858,"[Terminal 3, near Gate A13 (Toronto Pearson In...","[{'label': 'display', 'lat': 43.68522477548313...",43.685225,-79.623841,ON,52572cf511d2b76cc7c5a19c
3,Ice Bar,Bar,NaN,CA,Toronto,Canada,NaN,842,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.682519, 'lng':...",43.682519,-79.620245,ON,589622930a3d54567eca1682
4,Sky's The Limit - Lolo's Bar & Restaurant,None,NaN,CA,Grand Case,Canada,NaN,822,"[Grand Case, Canada]","[{'label': 'display', 'lat': 43.684174, 'lng':...",43.684174,-79.622541,NaN,4d2cea9039a35481ad88b320
5,Yorkville Bar & Grill,Sandwich Place,Pearson T3 Gate C24,CA,Mississauga,Canada,Airport Rd,1018,"[Pearson T3 Gate C24 (Airport Rd), Mississauga...","[{'label': 'display', 'lat': 43.68548593010033...",43.685486,-79.620924,ON,4bdf5bdac3392d7ff4479cd5
6,Playwright Bar & Grill,Restaurant,Toronto Pearson International Airport (YYZ),CA,Mississauga,Canada,NaN,828,"[Toronto Pearson International Airport (YYZ), ...","[{'label': 'display', 'lat': 43.68248529350422...",43.682485,-79.620415,ON,4b803e53f964a520346030e3
7,On A Plane To St Maarten/St Barts!,Plane,NaN,CA,NaN,Canada,NaN,1059,[Canada],NaN,43.684419,-79.618793,NaN,50718a05e4b09822b92c980a


In [169]:
#visualize all returns
dataframe_filtered.name

0                            Toast! Cafe & Bar
1                        Hogtown Bar and Grill
2                             Nobel Burger Bar
3                                      Ice Bar
4    Sky's The Limit - Lolo's Bar & Restaurant
5                        Yorkville Bar & Grill
6                       Playwright Bar & Grill
7           On A Plane To St Maarten/St Barts!
Name: name, dtype: object

In [171]:
#Plot of bars near airport
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto YYZ',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [207]:
#calculation of numbe of locations for varying radial search for each of the two compared neighborhoods YYZ, Billy Bishop
address = 'YYZ'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)



43.678523999999996 -79.62912913064454


In [208]:
search_query = 'Bar'
radius = 250

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe0 = json_normalize(venues)
dataframe0.shape
search_query = 'Bar'
radius = 450

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(venues)
dataframe1.shape

radius = 650

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues)

radius = 850

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe3 = json_normalize(venues)

print ('Toronto Pierson Airport')
print ('250 radius')
print (dataframe0.shape)
print ('450 radius')
print (dataframe1.shape)
print ('650 radius')
print (dataframe2.shape)
print ('850 radius')
print (dataframe3.shape)

Toronto Pierson Airport
250 radius
(0, 0)
450 radius
(0, 0)
650 radius
(3, 16)
850 radius
(8, 16)


In [213]:
#calculation of numbe of locations for varying radial search for each of the two compared neighborhoods YYZ, Billy Bishop
address = '2 Eireann Quay, Toronto, ON M5V 1A1, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)



43.6322406 -79.3963494


In [214]:
search_query = 'Bar'
radius = 250

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe0 = json_normalize(venues)
dataframe0.shape

search_query = 'Bar'
radius = 450

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(venues)
dataframe1.shape

radius = 650

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues)

radius = 850

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, 75)
#Get the results and transform into dataframe
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe3 = json_normalize(venues)

print ('Billy Bishop Airport')
print ('250 radius')
print (dataframe0.shape)
print ('450 radius')
print (dataframe1.shape)
print ('650 radius')
print (dataframe2.shape)
print ('850 radius')
print (dataframe3.shape)

Billy Bishop Airport
250 radius
(1, 16)
450 radius
(2, 16)
650 radius
(4, 16)
850 radius
(11, 18)


In [217]:
# End Capstone